In [1]:
ls ../../sample_input/

events.txt


In [2]:
ls ../../input

input.txt


In [3]:
from pyspark import SparkContext
import json
from datetime import datetime, timedelta

In [4]:
#path = "../../sample_input/events.txt"
path = "../../input/input.txt"
sc = None
sc = SparkContext()
csv_lines = sc.textFile(path)

In [5]:
csv_lines.collect()

[u'{"last_name": "Hopkins", "event_time": "2017-02-16T15:35:25.398Z", "verb": "NEW", "key": "27c0ac99f3d5", "adr_state": "NM", "adr_city": "Clarkstad", "type": "CUSTOMER"}',
 u'{"event_time": "2017-02-19T15:35:25.413Z", "tags": {"some key": "some value"}, "verb": "NEW", "key": "615624619603", "customer_id": "27c0ac99f3d5", "type": "SITE_VISIT"}',
 u'{"camera_make": "Kodak", "event_time": "2017-02-19T15:35:25.447Z", "camera_model": "Pixipro S-1", "verb": "UPLOAD", "key": "60d375151383", "customer_id": "27c0ac99f3d5", "type": "IMAGE"}',
 u'{"total_amount": "39.47 USD", "event_time": "2017-02-19T15:35:25.478Z", "verb": "NEW", "key": "18652c9e67fe", "customer_id": "27c0ac99f3d5", "type": "ORDER"}',
 u'{"last_name": "Marshall", "event_time": "2017-02-19T15:35:25.496Z", "verb": "NEW", "key": "27c0ac99f3d5", "adr_state": "WV", "adr_city": "South Theresa", "type": "CUSTOMER"}',
 u'{"event_time": "2017-02-10T15:35:25.517Z", "tags": {"some key": "some value"}, "verb": "NEW", "key": "07d45745fc12

In [6]:
list_of_lines = csv_lines.map(lambda line: line.replace('[','').strip(',').replace(']',''))
list_of_lines.collect()

[u'{"last_name": "Hopkins", "event_time": "2017-02-16T15:35:25.398Z", "verb": "NEW", "key": "27c0ac99f3d5", "adr_state": "NM", "adr_city": "Clarkstad", "type": "CUSTOMER"}',
 u'{"event_time": "2017-02-19T15:35:25.413Z", "tags": {"some key": "some value"}, "verb": "NEW", "key": "615624619603", "customer_id": "27c0ac99f3d5", "type": "SITE_VISIT"}',
 u'{"camera_make": "Kodak", "event_time": "2017-02-19T15:35:25.447Z", "camera_model": "Pixipro S-1", "verb": "UPLOAD", "key": "60d375151383", "customer_id": "27c0ac99f3d5", "type": "IMAGE"}',
 u'{"total_amount": "39.47 USD", "event_time": "2017-02-19T15:35:25.478Z", "verb": "NEW", "key": "18652c9e67fe", "customer_id": "27c0ac99f3d5", "type": "ORDER"}',
 u'{"last_name": "Marshall", "event_time": "2017-02-19T15:35:25.496Z", "verb": "NEW", "key": "27c0ac99f3d5", "adr_state": "WV", "adr_city": "South Theresa", "type": "CUSTOMER"}',
 u'{"event_time": "2017-02-10T15:35:25.517Z", "tags": {"some key": "some value"}, "verb": "NEW", "key": "07d45745fc12

In [7]:
list_of_lines.first()

u'{"last_name": "Hopkins", "event_time": "2017-02-16T15:35:25.398Z", "verb": "NEW", "key": "27c0ac99f3d5", "adr_state": "NM", "adr_city": "Clarkstad", "type": "CUSTOMER"}'

In [8]:
data = list_of_lines.map(lambda line: json.dumps(json.loads(line)))
data.collect()

['{"last_name": "Hopkins", "event_time": "2017-02-16T15:35:25.398Z", "verb": "NEW", "key": "27c0ac99f3d5", "adr_state": "NM", "adr_city": "Clarkstad", "type": "CUSTOMER"}',
 '{"event_time": "2017-02-19T15:35:25.413Z", "tags": {"some key": "some value"}, "verb": "NEW", "key": "615624619603", "customer_id": "27c0ac99f3d5", "type": "SITE_VISIT"}',
 '{"camera_make": "Kodak", "event_time": "2017-02-19T15:35:25.447Z", "camera_model": "Pixipro S-1", "verb": "UPLOAD", "key": "60d375151383", "customer_id": "27c0ac99f3d5", "type": "IMAGE"}',
 '{"total_amount": "39.47 USD", "event_time": "2017-02-19T15:35:25.478Z", "verb": "NEW", "key": "18652c9e67fe", "customer_id": "27c0ac99f3d5", "type": "ORDER"}',
 '{"last_name": "Marshall", "event_time": "2017-02-19T15:35:25.496Z", "verb": "NEW", "key": "27c0ac99f3d5", "adr_state": "WV", "adr_city": "South Theresa", "type": "CUSTOMER"}',
 '{"event_time": "2017-02-10T15:35:25.517Z", "tags": {"some key": "some value"}, "verb": "NEW", "key": "07d45745fc12", "cu

In [9]:
data.first()

'{"last_name": "Hopkins", "event_time": "2017-02-16T15:35:25.398Z", "verb": "NEW", "key": "27c0ac99f3d5", "adr_state": "NM", "adr_city": "Clarkstad", "type": "CUSTOMER"}'

In [10]:
def customer_key_to_id(line):
    event = json.loads(line)
    if event['type'] == 'CUSTOMER':
        customer_id = event['key']
    else:
        customer_id = event['customer_id']
    return customer_id

In [11]:
grouped_lines = list_of_lines.groupBy(lambda line: customer_key_to_id(line))
byCustomer = grouped_lines.map(lambda x: (x[0], list(x[1])))
byCustomer.collect()

[(u'313c194a650d',
  [u'{"last_name": "Riley", "event_time": "2017-02-17T15:35:28.140Z", "verb": "NEW", "key": "313c194a650d", "adr_state": "NE", "adr_city": "Jeremyshire", "type": "CUSTOMER"}',
   u'{"event_time": "2017-02-22T15:35:28.159Z", "tags": {"some key": "some value"}, "verb": "NEW", "key": "65fb21ff2053", "customer_id": "313c194a650d", "type": "SITE_VISIT"}',
   u'{"camera_make": "Nikon", "event_time": "2017-02-22T15:35:28.192Z", "camera_model": "Coolpix A10", "verb": "UPLOAD", "key": "0378ba0fdfb4", "customer_id": "313c194a650d", "type": "IMAGE"}',
   u'{"total_amount": "47.56 USD", "event_time": "2017-02-23T15:35:28.225Z", "verb": "NEW", "key": "33eae58652fe", "customer_id": "313c194a650d", "type": "ORDER"}',
   u'{"last_name": "Cross", "event_time": "2017-02-20T15:35:28.240Z", "verb": "NEW", "key": "313c194a650d", "adr_state": "CO", "adr_city": "Rothfurt", "type": "CUSTOMER"}',
   u'{"event_time": "2017-02-19T15:35:28.254Z", "tags": {"some key": "some value"}, "verb": "NEW

In [12]:
def sort_events_by_datetime(events_list):
    time_event_tuples = [(json.loads(event)['event_time'],event) for event in events_list]
    time_event_tuples.sort()
    return time_event_tuples

In [13]:
grouped_lines = list_of_lines.groupBy(lambda line: customer_key_to_id(line))
byCustomer = grouped_lines.map(lambda x: ( x[0],sort_events_by_datetime(list(x[1])) ) )
byCustomer.collect()

[(u'313c194a650d',
  [(u'2017-02-10T15:35:28.435Z',
    u'{"last_name": "Kane", "event_time": "2017-02-10T15:35:28.435Z", "verb": "NEW", "key": "313c194a650d", "adr_state": "AZ", "adr_city": "New Philip", "type": "CUSTOMER"}'),
   (u'2017-02-10T15:35:28.518Z',
    u'{"total_amount": "32.18 USD", "event_time": "2017-02-10T15:35:28.518Z", "verb": "NEW", "key": "8f2762947cff", "customer_id": "313c194a650d", "type": "ORDER"}'),
   (u'2017-02-11T15:35:28.331Z',
    u'{"last_name": "Bailey", "event_time": "2017-02-11T15:35:28.331Z", "verb": "NEW", "key": "313c194a650d", "adr_state": "ME", "adr_city": "Wintersshire", "type": "CUSTOMER"}'),
   (u'2017-02-11T15:35:28.546Z',
    u'{"last_name": "Meyer", "event_time": "2017-02-11T15:35:28.546Z", "verb": "NEW", "key": "313c194a650d", "adr_state": "SC", "adr_city": "South Cheryl", "type": "CUSTOMER"}'),
   (u'2017-02-13T15:35:28.454Z',
    u'{"event_time": "2017-02-13T15:35:28.454Z", "tags": {"some key": "some value"}, "verb": "NEW", "key": "2b5d39

In [16]:
def parse_event_time(event_time):
    dt,tm = event_time.strip('Z').split('T')
    t = ':'.join(tm.split(':')[:3])
    hms = t.split('.')[0]
    ymd_hms = ' '.join([dt,hms])
    return dt,t,ymd_hms

def dt_to_day(dt,fmt="%Y-%m-%d"):
    return datetime.strptime(dt,fmt).strftime('%A')

def str_to_dt_var(str_dt,fmt="%Y-%m-%d %H:%M:%S"):
    return datetime.strptime(str_dt,fmt)

def dt_var_to_str(dt_var,fmt="%Y-%m-%d %H:%M:%S"):
    return datetime.strftime(dt_var,fmt)

def week_start_end(week_range):
    list_of_week_start_end = []
    start = week_range[0]
    end = dt_var_to_str(str_to_dt_var(start,fmt="%Y-%m-%d")+timedelta(days=6),
                        fmt="%Y-%m-%d")
    list_of_week_start_end.append((start,end))
    while end < week_range[-1]:
        start = dt_var_to_str(str_to_dt_var(end,fmt="%Y-%m-%d")+timedelta(days=1),
                        fmt="%Y-%m-%d")
        end = dt_var_to_str(str_to_dt_var(start,fmt="%Y-%m-%d")+timedelta(days=6),
                        fmt="%Y-%m-%d")
        list_of_week_start_end.append((start,end))    
    return list_of_week_start_end

def week_is_number(date,list_of_week_start_end):
    dt_var = str_to_dt_var(date,fmt="%Y-%m-%d")
    week_number = [i for i in range(len(list_of_week_start_end)) 
                   if dt_var >= str_to_dt_var(list_of_week_start_end[i][0],fmt="%Y-%m-%d")
                     and dt_var <= str_to_dt_var(list_of_week_start_end[i][1],fmt="%Y-%m-%d")][0]
    return week_number

def sessionize_datetime_data(session_data,minute_window=30,
                             week_def={'Sunday':0,
                                   'Monday':1,
                                   'Tuesday':2,
                                   'Wednesday':3,
                                   'Thursday':4,
                                   'Friday':5,
                                   'Saturday':6},
                             hdr=['date','day','session_start','prior_session_start'
                                  ,'minutes_since_prior_session']):



    week_numbers = [0]
    initial_day = dict(zip(hdr,session_data[0][0]))['day']
    initial_date = dict(zip(hdr,session_data[0][0]))['date']
    final_date = dict(zip(hdr,session_data[-1][0]))['date']
    week_range = [initial_date,final_date]
    n = 0
    session_numbers = [n]
    if initial_day != 'Sunday':
        initial_dt_var = str_to_dt_var(initial_date,fmt="%Y-%m-%d")-timedelta(days=week_def[initial_day]-1)
        week_range[0] = dt_var_to_str(initial_dt_var,fmt="%Y-%m-%d")                           
    list_of_week_start_end = week_start_end(week_range)
    for i in range(1,len(session_data)):
        session_dict = dict(zip(hdr,session_data[i][0]))
        minutes_since_prior_session = session_dict['minutes_since_prior_session']
        if minutes_since_prior_session <= minute_window:
            session_numbers.append(n)
        else:
            n += 1
            session_numbers.append(n)            
        date = session_dict['date']
        week_number = week_is_number(date,list_of_week_start_end)
        week_numbers.append(week_number)

    return week_numbers,session_numbers


    
    

def sessionize(time_event_tuples):
    session_data = []
    initial_event_time,initial_event = time_event_tuples[0]
    initial_dt,initial_t,initial_ymd_hms = parse_event_time(initial_event_time)
    initial_d = dt_to_day(initial_dt)
    session_data.append(([initial_dt,initial_d,initial_t,None,None],initial_event))
    for i in range(1,len(time_event_tuples)):
        event_time,event = time_event_tuples[i]
        prior_event_time,prior_event = time_event_tuples[i-1]
        dt,t,ymd_hms = parse_event_time(event_time)
        d = dt_to_day(dt)
        prior_dt,prior_t,prior_ymd_hms = parse_event_time(prior_event_time)
        seconds_since_prior_t = (str_to_dt_var(ymd_hms,fmt="%Y-%m-%d %H:%M:%S")
                                 -str_to_dt_var(prior_ymd_hms,fmt="%Y-%m-%d %H:%M:%S")).seconds
        minutes_since_prior_t = (seconds_since_prior_t % 3600) // 60
        session_data.append(([dt,d,t,prior_t,minutes_since_prior_t],event))
    
    week_numbers,session_numbers = sessionize_datetime_data(session_data)
    events = [session_events[1] for session_events in session_data]
    return [zip(week_numbers,session_numbers,events)]


In [17]:
byCustomerSession = byCustomer.map(lambda x: (x[0],sessionize(x[1])) )
byCustomerSession.collect()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 12.0 failed 1 times, most recent failure: Lost task 1.0 in stage 12.0 (TID 19, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 172, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 167, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 263, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "<ipython-input-17-030a7fc276ba>", line 1, in <lambda>
  File "<ipython-input-16-0ef51d1bff5c>", line 91, in sessionize
  File "<ipython-input-16-0ef51d1bff5c>", line 11, in str_to_dt_var
  File "/opt/conda/envs/python2/lib/python2.